# Run the CICI SAFE vSDX Using Chameleon

#### Configure the Envrionment

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@TACC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/pruth-chameleon-jupyter


#Set the working directory
export WORKING_DIR=${HOME}/work/CICI-SAFE/Chameleon-Jupyter-Tridentcom
export UNIVERSAL_INFO_FILE=${WORKING_DIR}/universal_info
export SDX_INFO_FILE=${WORKING_DIR}/sdx_info
echo "" > $SDX_INFO_FILE

export SDX_NETWORK='sharednet1'
export SDX_NODE_NAME=${USERNAME}-sdx
export SDX_STACK_NAME=${USERNAME}-sdx

echo SDX_NODE_NAME $SDX_NODE_NAME >> $SDX_INFO_FILE
echo SDX_STACK_NAME $SDX_STACK_NAME >> $SDX_INFO_FILE



#RIAK Info
export RIAK_INFO_FILE=${WORKING_DIR}/riak_info
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`
echo RIAK_PUBLIC_IP: $RIAK_PUBLIC_IP

#### Create Lease (if necessary)

In [ ]:
#Check of universal reservation
if [ -f "${UNIVERSAL_INFO_FILE}" ]; then
   LEASE_NAME=`cat $UNIVERSAL_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
   echo $LEASE_NAME
   echo LEASE_NAME $LEASE_NAME >> $RIAK_INFO_FILE
fi

if  [ -z $LEASE_NAME ]; then
  
    echo No universal reservation. Creating new reservation

# --start-date "2019-02-10 17:50" \
# --end-date "2019-02-11 12:00" \
START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+1 day" +'%F %H:%M'`
LEASE_NAME=${USERNAME}"-sdx-lease"
echo LEASE_NAME $LEASE_NAME >> $SDX_INFO_FILE

NODE_MIN=1
NODE_MAX=1

NODE_TYPE='compute_haswell'
#NODE_TYPE='compute_skylake'

echo Creating Lease ${LEASE_NAME}
blazar lease-create \
   --physical-reservation min=${NODE_MIN},max=${NODE_MAX},resource_properties='["=", "$node_type", "'${NODE_TYPE}'"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${LEASE_NAME}

fi

echo LEASE_NAME: $LEASE_NAME
cat  $RIAK_INFO_FILE

In [ ]:
RESERVATION_ID=`blazar lease-show --format value -c id ${LEASE_NAME}`
echo RESERVATION_ID $RESERVATION_ID

RESERVATION_RESOURCE_ID=`blazar lease-show --format value -c reservations ${RESERVATION_ID} | grep \"id\" | cut -d "\"" -f 4`
echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID

#blazar lease-show ${RESERVATION_ID}

#Poll for node to be ready for ssh
while [[ true ]]
do 
	RESERVATION_STATUS=`blazar lease-show --format value -c status ${RESERVATION_ID}`
    
    if [[ "${RESERVATION_STATUS}" == "ACTIVE" ]]; then break; fi
    
    echo Reservation is ${RESERVATION_STATUS}. retrying in 5 seconds.
    sleep 5
done

echo RESERVATION_STATUS ${RESERVATION_STATUS}

echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID >> $SDX_INFO_FILE

cat $SDX_INFO_FILE


#### Start SDX Server

In [ ]:
RESERVATION_RESOURCE_ID=`cat $SDX_INFO_FILE | grep RESERVATION_RESOURCE_ID | cut -d " " -f 2`
SDX_NODE_NAME=`cat $SDX_INFO_FILE | grep SDX_NODE_NAME | cut -d " " -f 2`
SDX_STACK_NAME=`cat $SDX_INFO_FILE | grep SDX_STACK_NAME | cut -d " " -f 2`

RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`


echo Creating SDX server. This will take several minutes!
openstack stack create  --max-width 80 \
        --template sdx.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${RESERVATION_RESOURCE_ID}" \
        --parameter "network_name=${SDX_NETWORK}" \
        --parameter "sdx_node_name=${SDX_NODE_NAME}" \
        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
        --wait \
        ${SDX_STACK_NAME}

In [ ]:
SDX_NODE_NAME=`cat $SDX_INFO_FILE | grep SDX_NODE_NAME | cut -d " " -f 2`

echo SDX server creation complete! 
echo ${SDX_NODE_NAME} status `openstack server show  --format value -c status ${SDX_NODE_NAME}`


SDX_PUBLIC_IP=`openstack server show  --format value -c addresses ${SDX_NODE_NAME} | cut -d " " -f 2`
echo The SDX public IP is $SDX_PUBLIC_IP
echo SDX_PUBLIC_IP $SDX_PUBLIC_IP >> $SDX_INFO_FILE


In [ ]:
#SDX_PUBLIC_IP=`openstack server show --format value -c addresses ${SDX_NODE_NAME} | cut -d " " -f 2`
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`
echo SDX_PUBLIC_IP is $SDX_PUBLIC_IP

#Poll for node to be ready for ssh
while [[ true ]]
do 
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    exit
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo Node not ready for ssh. retrying in 5 seconds.
    sleep 5
done

echo SDX node ready for ssh

In [ ]:
#Tail log until complete. Note: YOU MUST MANUALLY STOP THIS TO CONTINUE!

SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`
echo SDX_PUBLIC_IP is $SDX_PUBLIC_IP



#ssh -i ${SSH_PRIVATE_KEY} \
#    -o UserKnownHostsFile=/dev/null \
#    -o StrictHostKeyChecking=no \
#    cc@${SDX_PUBLIC_IP} \
#    sudo tail /root/boot.log
    
    
#Poll for node to be ready for ssh
while [[ true ]]
do
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    sudo tail /root/boot.log | grep 'Boot Script Done!'
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo 'Not Done, Retrying in 30 sec'
    sleep 30
done

echo SDX Boot Script Done!

#### Configure the SDX node

In [ ]:
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`
echo SDX_PUBLIC_IP is $SDX_PUBLIC_IP

#Copy GENI pem file to node
REMOTE_GENI_CERT=/home/cc/geni.pem
GENI_CERT=${HOME}/work/geni-pruth.pem

echo Copying GENI cert to cc@$SDX_PUBLIC_IP:~/geni.pem
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    ${GENI_CERT} \
    cc@${SDX_PUBLIC_IP}:${REMOTE_GENI_CERT}
echo Done

In [ ]:
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`

LOCAL_SDX_CONFIG=${WORKING_DIR}/SDX/sdx.conf

REMOTE_SDX_CONFIG=/home/cc/sdx.conf
REMOTE_WORKING_DIR=/root
REMOTE_BIN_DIR=${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/bin

DATAPLANE_SLICE_NAME=${USERNAME}-sdx-dataplane

cat << EOF > $LOCAL_SDX_CONFIG
config {
safe=true
slicename="${DATAPLANE_SLICE_NAME}"
type="server"
sshkey="/root/.ssh/id_rsa"
safekey="sdx"
exogenipem="${REMOTE_GENI_CERT}"
exogenism="https://geni.renci.org:11443/orca/xmlrpc"
ipprefix="192.168.128.1/24"
riakserver="${RIAK_PUBLIC_IP}"
serverinslice=false
safeserver="${SDX_PUBLIC_IP}"
plexusserver="${SDX_PUBLIC_IP}"
serverurl="http://0.0.0.0:8888/"
scriptsdir="${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/src/main/resources/scripts/"
clientsites:"UFL:UNF"
controllersite:"UFL (Gainesville, FL USA) XO Rack"
serversite:"RENCI (Chapel Hill, NC USA) XO Rack"
}
EOF

echo Copying SDX config to cc@$SDX_PUBLIC_IP
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    $LOCAL_SDX_CONFIG \
    cc@${SDX_PUBLIC_IP}:${REMOTE_SDX_CONFIG}
echo Done

#### Start the Dataplane Slice

In [ ]:
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`

echo Start SDX dataplane Slice
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    sudo ${REMOTE_BIN_DIR}/SafeSdxSliceServer -c $REMOTE_SDX_CONFIG  
echo Slice Creation Done!

#### Start the SDX Server

In [ ]:
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`

echo Start the SDX Server
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    "sudo pkill java; sudo ${REMOTE_BIN_DIR}/SafeSdxServer -c $REMOTE_SDX_CONFIG &"

# Please Cleanup Your Resources!!!

#### Delete Dataplane Slice

In [ ]:
#need to install exogeni phython script in the container
#python delete ${DATAPLANE_SLICE_NAME}

echo Delete SDX Slice

echo $REMOTE_SDX_CONFIG

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxSliceServer -c $REMOTE_SDX_CONFIG -d"
    
echo Slice Deletion Done

#### Clean Up SDX Chaeleon Stack and Reservation

In [ ]:
SDX_STACK_NAME=`cat $SDX_INFO_FILE | grep SDX_STACK_NAME | cut -d " " -f 2`
openstack stack delete --wait -y $SDX_STACK_NAME

LEASE_NAME=`cat $SDX_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
blazar lease-delete $LEASE_NAME

cat $SDX_INFO_FILE
mv $SDX_INFO_FILE ${SDX_INFO_FILE}.old